In [1]:
!git clone https://github.com/Muennighoff/performers.git
!cd performers; pip install -q -e '.[dev]'

!pip install -q datasets
!pip install -q ml_collections
!pip install -q --upgrade jax jaxlib

Cloning into 'performers'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 56020 (delta 0), reused 2 (delta 0), pack-reused 56013
Receiving objects: 100% (56020/56020), 42.40 MiB | 28.19 MiB/s, done.
Resolving deltas: 100% (39274/39274), done.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.9MB 9.1MB/s 
     |████████████████████████████████| 890kB 41.7MB/s 
     |████████████████████████████████| 47.4MB 87kB/s 
     |████████████████████████████████| 112kB 49.7MB/s 
     |████████████████████████████████| 31.9MB 197kB/s 
     |████████████████████████████████| 81kB 12.0MB/s 
     |████████████████████████████████| 163kB 55.6MB/s 
     |████████████████████████████████| 102kB 13.0MB/s 
     |████████████████████████████████| 1.1MB 51.1MB/s 
     |██████████████████████████

In [2]:
import os
import sys
import tensorflow as tf
import math
from datasets import load_dataset

module_path = os.path.abspath(os.path.join('./performers/src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

from transformers.models.distilbert import DistilBertConfig, DistilBertForSequenceClassification
from transformers.models.distilbert import TFDistilBertForSequenceClassification
from transformers.models.bert import BertConfig, TFBertForSequenceClassification

##### IMDB Data Preparation

In [3]:
"""Input pipeline for the imdb dataset."""

from absl import logging
import numpy as np
import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

AUTOTUNE = tf.data.experimental.AUTOTUNE


def preprocess_dataset(file_path, batch_size):
  """Preprocess dataset."""
  tf.logging.info(file_path)
  sel_cols = ['Source', 'Target']
  col_defaults = [tf.string, tf.int32]
  ds = tf.data.experimental.make_csv_dataset([file_path],
                                             batch_size,
                                             column_defaults=col_defaults,
                                             select_columns=sel_cols,
                                             field_delim=',',
                                             header=True,
                                             shuffle=False,
                                             num_epochs=1)
  ds = ds.unbatch()
  return ds


def get_imdb_dataset():
  """Get dataset from  imdb tfds. converts into src/tgt pairs."""
  data = tfds.load('imdb_reviews')
  train_raw = data['train']
  valid_raw = data['test']
  test_raw = data['test']
  # use test set for validation because IMDb doesn't have val set.
  # Print an example.
  logging.info('Data sample: %s', next(iter(tfds.as_numpy(train_raw.skip(4)))))

  def adapt_example(example):
    return {'Source': example['text'], 'Target': example['label']}

  train = train_raw.map(adapt_example)
  valid = valid_raw.map(adapt_example)
  test = test_raw.map(adapt_example)

  return train, valid, test


def get_yelp_dataset():
  """Get dataset from yelp tfds. converts into src/tgt pairs."""
  data = tfds.load('yelp_polarity_reviews')
  train_raw = data['train']
  valid_raw = data['test']
  test_raw = data['test']
  # use test set for validation because yelp doesn't have val set.
  # Print an example.
  logging.info('Data sample: %s', next(iter(tfds.as_numpy(train_raw.skip(4)))))

  def adapt_example(example):
    return {'Source': example['text'], 'Target': example['label']}

  train = train_raw.map(adapt_example)
  valid = valid_raw.map(adapt_example)
  test = test_raw.map(adapt_example)

  return train, valid, test


def get_agnews_dataset():
  """Get dataset from  agnews tfds. converts into src/tgt pairs."""
  data = tfds.load('ag_news_subset')
  train_raw = data['train']
  valid_raw = data['test']
  test_raw = data['test']
  # use test set for validation because agnews doesn't have val set.
  # Print an example.
  logging.info('Data sample: %s', next(iter(tfds.as_numpy(train_raw.skip(4)))))

  def adapt_example(example):
    return {'Source': example['description'], 'Target': example['label']}

  train = train_raw.map(adapt_example)
  valid = valid_raw.map(adapt_example)
  test = test_raw.map(adapt_example)

  return train, valid, test


def get_tc_datasets(n_devices,
                    task_name,
                    data_dir=None,
                    batch_size=256,
                    fixed_vocab=None,
                    max_length=512,
                    tokenizer='char'):
  """Get text classification datasets."""
  if batch_size % n_devices:
    raise ValueError("Batch size %d isn't divided evenly by n_devices %d" %
                     (batch_size, n_devices))

  if task_name == 'imdb_reviews':
    train_dataset, val_dataset, test_dataset = get_imdb_dataset()
  elif task_name == 'yelp_reviews':
    train_dataset, val_dataset, test_dataset = get_yelp_dataset()
  elif task_name == 'agnews':
    train_dataset, val_dataset, test_dataset = get_agnews_dataset()
  else:
    train_path = data_dir + task_name + '_train.tsv'
    val_path = data_dir + task_name + '_val.tsv'
    test_path = data_dir + task_name + '_test.tsv'

    train_dataset = preprocess_dataset(train_path, batch_size)
    val_dataset = preprocess_dataset(val_path, batch_size)
    test_dataset = preprocess_dataset(test_path, batch_size)

  tf.logging.info('Finished preprocessing')

  tf.logging.info(val_dataset)

  if tokenizer == 'char':
    logging.info('Using char/byte level vocab')
    encoder = tfds.deprecated.text.ByteTextEncoder() ####################
  else:
    if fixed_vocab is None:
      tf.logging.info('Building vocab')
      # build vocab
      vocab_set = set()
      tokenizer = tfds.deprecated.text.Tokenizer()
      for i, data in enumerate(train_dataset):
        examples = data['Source']
        examples = tokenizer.tokenize(examples.numpy())
        examples = np.reshape(examples, (-1)).tolist()
        vocab_set.update(examples)
        if i % 1000 == 0:
          tf.logging.info('Processed {}'.format(i))
      tf.logging.info(len(vocab_set))
      vocab_set = list(set(vocab_set))
      tf.logging.info('Finished processing vocab size={}'.format(
          len(vocab_set)))
    else:
      vocab_set = list(set(fixed_vocab))
    encoder = tfds.deprecated.text.TokenTextEncoder(vocab_set)

  def tf_encode(x):
    result = tf.py_function(lambda s: tf.constant(encoder.encode(s.numpy())), [
        x,
    ], tf.int32)
    result.set_shape([None])
    return result

  def tokenize(d):
    return {
        'input_ids': tf_encode(d['Source'])[:max_length], ################
        'targets': d['Target']
    }

  train_dataset = train_dataset.map(tokenize, num_parallel_calls=AUTOTUNE)
  val_dataset = val_dataset.map(tokenize, num_parallel_calls=AUTOTUNE)
  test_dataset = test_dataset.map(tokenize, num_parallel_calls=AUTOTUNE)

  max_shape = {'input_ids': [max_length], 'targets': []}
  train_dataset = train_dataset.shuffle(
      buffer_size=256, reshuffle_each_iteration=True).padded_batch(
          batch_size, padded_shapes=max_shape, drop_remainder=True) ######## Added drop_remainder
  val_dataset = val_dataset.padded_batch(batch_size, padded_shapes=max_shape, drop_remainder=True)
  test_dataset = test_dataset.padded_batch(batch_size, padded_shapes=max_shape, drop_remainder=True)

  return train_dataset, val_dataset, test_dataset, encoder

In [4]:
# LRA Config

import ml_collections

def get_config():
  """Get the default hyperparameter configuration."""
  config = ml_collections.ConfigDict()
  config.batch_size = 8 #32            # At BS 8, 3125 in train & val dataset
  config.eval_frequency = 100
  config.num_train_steps = 20000
  config.num_eval_steps = -1
  config.learning_rate = 0.05
  config.weight_decay = 1e-1
  config.max_target_length = 200
  config.max_eval_target_length = 200
  config.sampling_temperature = 0.6
  config.sampling_top_k = 20
  config.max_predict_token_length = 50
  config.save_checkpoints = True
  config.restore_checkpoints = True
  config.checkpoint_freq = 10000
  config.random_seed = 0
  config.prompt = ""
  config.factors = "constant * linear_warmup * rsqrt_decay"
  config.warmup = 8000
  config.classifier_pool = "CLS"

  config.max_length = 1000 #4K in paper?

  config.emb_dim = 256
  config.num_heads = 4
  config.num_layers = 4
  config.qkv_dim = 256
  config.mlp_dim = 1024

  config.trial = 0  # dummy for repeated runs.
  return config

In [5]:
# Get data
lra_config = get_config()

train_ds, eval_ds, test_ds, encoder = get_tc_datasets(
    n_devices=1, #jax.local_device_count(),
    task_name="imdb_reviews", # Used in paper
    data_dir="/content/",
    batch_size=lra_config.batch_size,
    fixed_vocab=None, # If we already have a vocab
    max_length=lra_config.max_length-1) # Use fixed max l of 4K > -2 to acount for <s> & </s>

INFO:absl:No config specified, defaulting to first: imdb_reviews/plain_text
INFO:absl:Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: imdb_reviews/plain_text/1.0.0
INFO:absl:Load dataset info from /tmp/tmpucq1wvvitfds
INFO:absl:Field info.config_name from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.config_description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.citation from disk and from code do not match. Keeping the one from code.
INFO:absl:Generating dataset imdb_reviews (/root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0)


INFO:absl:Downloading http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz into /root/tensorflow_datasets/downloads/ai.stanfor.edu_amaas_sentime_aclImdb_v1PaujRp-TxjBWz59jHXsMDm5WiexbxzaFQkEnXc3Tvo8.tar.gz.tmp.dda099a09e6847e2b76bcf1c2e4940b6...
INFO:absl:Generating split train


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHTRFRO/imdb_reviews-train.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHTRFRO/imdb_reviews-train.tfrecord. Shard lengths: [25000]
INFO:absl:Generating split test


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHTRFRO/imdb_reviews-test.tfrecord


...

INFO:absl:Data sample: {'label': 1, 'text': b'As others have mentioned, all the women that go nude in this film are mostly absolutely gorgeous. The plot very ably shows the hypocrisy of the female libido. When men are around they want to be pursued, but when no "men" are around, they become the pursuers of a 14 year old boy. And the boy becomes a man really fast (we should all be so lucky at this age!). He then gets up the courage to pursue his true love.'}


INFO:tensorflow:Finished preprocessing


INFO:tensorflow:Finished preprocessing


INFO:tensorflow:<MapDataset shapes: {Source: (), Target: ()}, types: {Source: tf.string, Target: tf.int64}>


INFO:tensorflow:<MapDataset shapes: {Source: (), Target: ()}, types: {Source: tf.string, Target: tf.int64}>
INFO:absl:Using char/byte level vocab


In [6]:
# Create attention masks

def add_attn_mask(example):
    """
    Add attention_mask to inputs; Don't attend to 0s
    """

    # Get a boolean wherever input_ids > 0; then cast to ints for attn_mask
    example["attention_mask"] = tf.cast((example["input_ids"] > 0)[...], tf.int32)

    return example

train_ds = train_ds.map(add_attn_mask)
eval_ds = eval_ds.map(add_attn_mask)
test_ds = test_ds.map(add_attn_mask)

In [7]:
# Add BOS (& EOS)

vocab_size = encoder.vocab_size
print('Vocab Size: ', vocab_size)

bos = tf.constant(vocab_size, shape=(lra_config.batch_size,1))
eos = tf.constant(vocab_size+1, shape=(lra_config.batch_size,1))

attn_addon = tf.constant(1, shape=(lra_config.batch_size,1))

def add_tokens(example):
    """
    Add <s> & </s> tokens // bos & eos

    Only adding bos for now, as they do not add eos token in the original & we alrdy have it padded
    """

    example["input_ids"] = tf.concat([bos, example["input_ids"]], axis=1)

    # Add 1 to attn_mask
    example["attention_mask"] = tf.concat([attn_addon, example["attention_mask"]], axis=1)

    return example


train_ds = train_ds.map(add_tokens)
eval_ds = eval_ds.map(add_tokens)
test_ds = test_ds.map(add_tokens)

Vocab Size:  257


In [8]:
# Switch to 2.X
import tensorflow as tf

# Overfit on one batch
#train_ds = eval_ds.take(1)
#train_ds = train_ds.repeat()
#eval_ds = eval_ds.take(1)

train_ds = train_ds.repeat()

##### Modeling & Training

In [9]:
# Load & adjust hgface config
#config = RobertaConfig.from_pretrained("roberta-base")
config = DistilBertConfig()

print(config)

config.attention_type='performer' # Normal softmax
#config.performer = True
config.vocab_size = vocab_size + 1
config.max_position_embeddings = lra_config.max_length + 1 # Need more than our seq_len as it seems the first isnt counted

config.bos_token_id = vocab_size
config.eos_token_id = vocab_size + 1 # Not used for now
config.pad_token_id = 0

# Change to config dims
config.n_heads = lra_config.num_heads
#config.num_attention_heads = lra_config.num_heads # 4
config.n_layers = lra_config.num_layers
#config.num_hidden_layers = lra_config.num_layers # 4
config.intermediate_size = lra_config.mlp_dim # 1024
config.hidden_dim = lra_config.qkv_dim
#config.hidden_size = lra_config.qkv_dim # 256 # Is this the same?


print(config)

DistilBertConfig {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "attention_type": "softmax",
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "performer_attention_config": null,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "vocab_size": 30522
}

DistilBertConfig {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "attention_type": "performer",
  "bos_token_id": 257,
  "dim": 768,
  "dropout": 0.1,
  "eos_token_id": 258,
  "hidden_dim": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "max_position_embeddings": 1001,
  "model_type": "distilbert",
  "n_heads": 4,
  "n_layers": 4,
  "pad_token_id": 0,
  "performer_attention_config": null,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "vocab_size": 258
}



In [10]:
import tensorflow_addons as tfa

model = TFDistilBertForSequenceClassification(config)
#model = TFRobertaForSequenceClassification(config)
#model = TFBertForSequenceClassification(config)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
                from_logits=True, reduction=tf.keras.losses.Reduction.NONE
            )

train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

optimizer = tf.keras.optimizers.Adam(lra_config.learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
#optimizer = tfa.optimizers.AdamW(weight_decay=0.01, learning_rate=lra_config.learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)#lra_config.weight_decay

In [11]:
def create_learning_rate_scheduler(
    factors='constant * linear_warmup * rsqrt_decay',
    base_learning_rate=0.5,
    warmup_steps=1000,
    decay_factor=0.5,
    steps_per_decay=20000,
    steps_per_cycle=100000):
  """Creates learning rate schedule.
  Interprets factors in the factors string which can consist of:
  * constant: interpreted as the constant value,
  * linear_warmup: interpreted as linear warmup until warmup_steps,
  * rsqrt_decay: divide by square root of max(step, warmup_steps)
  * rsqrt_normalized_decay: divide by square root of max(step/warmup_steps, 1)
  * decay_every: Every k steps decay the learning rate by decay_factor.
  * cosine_decay: Cyclic cosine decay, uses steps_per_cycle parameter.
  Args:
    factors: string, factors separated by '*' that defines the schedule.
    base_learning_rate: float, the starting constant for the lr schedule.
    warmup_steps: int, how many steps to warm up for in the warmup schedule.
    decay_factor: float, the amount to decay the learning rate by.
    steps_per_decay: int, how often to decay the learning rate.
    steps_per_cycle: int, steps per cycle when using cosine decay.
  Returns:
    a function learning_rate(step): float -> {'learning_rate': float}, the
    step-dependent lr.
  """
  factors = [n.strip() for n in factors.split('*')]

  def step_fn(step):
    """Step to learning rate function."""
    ret = 1.0
    for name in factors:
      if name == 'constant':
        ret *= base_learning_rate
      elif name == 'linear_warmup':
        ret *= tf.math.minimum(1.0, step / warmup_steps)
      elif name == 'rsqrt_decay':
        # tf.math.sqrt does not accept int
        ret /= tf.math.sqrt(tf.cast(tf.math.maximum(step, warmup_steps), tf.float32))
      elif name == 'rsqrt_normalized_decay':
        ret *= tf.math.sqrt(warmup_steps)
        ret /= tf.math.sqrt(tf.cast(tf.math.maximum(step, warmup_steps), tf.float32))
      elif name == 'decay_every':
        ret *= (decay_factor**(step // steps_per_decay))
      elif name == 'cosine_decay':
        progress = tf.math.maximum(0.0,
                               (step - warmup_steps) / float(steps_per_cycle))
        ret *= tf.math.maximum(0.0,
                           0.5 * (1.0 + tf.math.cos(tf.constant(math.pi) * (progress % 1.0))))
      else:
        raise ValueError('Unknown factor %s.' % name)
    return tf.convert_to_tensor(ret, dtype=tf.float32)

  return step_fn

In [12]:
learning_rate_fn = create_learning_rate_scheduler(
                    factors=lra_config.factors,
                    base_learning_rate=lra_config.learning_rate,
                    warmup_steps=lra_config.warmup)

In [13]:
import time

epochs = 10 # > 20K
total_steps = 0 # Break at 20K as in original

for epoch in range(epochs):

    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (batch) in enumerate(train_ds):
        total_steps += 1
        with tf.GradientTape() as tape:
            logits = model(batch, training=True).logits # Logits for this minibatch
            loss_value = loss_fn(batch["targets"], logits)
            loss_value = tf.math.reduce_mean(loss_value)
        grads = tape.gradient(loss_value, model.trainable_weights)

        # Calculate lr based on schedule & change
        lr = learning_rate_fn(total_steps) / 4  # 16 instead of their 32 BS
        tf.keras.backend.set_value(optimizer.learning_rate, lr)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Update training metric.
        train_acc_metric.update_state(batch["targets"], logits)

        # Log every 200 batches.
        if (step) % 200 == 0:
            print("Current LR:", lr)
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )

        # Evaluate every 2000 steps
        if (total_steps - 1) % 2000 == 0:
            # Display metrics at the end of each epoch.
            train_acc = train_acc_metric.result()
            print("Training acc over last 2K steps: %.4f" % (float(train_acc),))

            # Reset training metrics after 2K steps
            train_acc_metric.reset_states()

            # Run a validation loop at the end of each epoch.
            for batch in eval_ds:
                val_logits = model(batch, training=False).logits
                # Update val metrics
                val_acc_metric.update_state(batch["targets"], val_logits)
            val_acc = val_acc_metric.result()
            val_acc_metric.reset_states()
            print("Validation acc: %.4f" % (float(val_acc),))
            print("Time taken: %.2fs" % (time.time() - start_time))

        if total_steps % 20000 == 0:
            print("20,000 steps finished")
            print("=====================")
            break
    if total_steps % 20000 == 0:
        break


Start of epoch 0
Current LR: tf.Tensor(1.7469281e-08, shape=(), dtype=float32)
Training loss (for one batch) at step 0: 0.6846
Training acc over last 2K steps: 0.5000
Validation acc: 0.5002
Time taken: 352.27s
Current LR: tf.Tensor(3.5113258e-06, shape=(), dtype=float32)
Training loss (for one batch) at step 200: 0.7152
Current LR: tf.Tensor(7.005182e-06, shape=(), dtype=float32)
Training loss (for one batch) at step 400: 0.6857
Current LR: tf.Tensor(1.0499038e-05, shape=(), dtype=float32)
Training loss (for one batch) at step 600: 0.6944
Current LR: tf.Tensor(1.3992894e-05, shape=(), dtype=float32)
Training loss (for one batch) at step 800: 0.6555
Current LR: tf.Tensor(1.7486751e-05, shape=(), dtype=float32)
Training loss (for one batch) at step 1000: 0.6871
Current LR: tf.Tensor(2.0980606e-05, shape=(), dtype=float32)
Training loss (for one batch) at step 1200: 0.6401
Current LR: tf.Tensor(2.4474464e-05, shape=(), dtype=float32)
Training loss (for one batch) at step 1400: 0.6701
Cur

In [ ]:
# Temporarily reaches 59.9% in-between which is on par with thr 64% reported (They took the best from 4 diff. runs in paper)